# Семинар 7. Инструменты для работы с языком

... или зачем нужна предобработка.

Раньше мы смотрели на светлую сторону анализа данных - построение моделей. Теперь попробуем глубже посмотреть на часть про предобработку данных. Задача предобработки особенно актуальна, если мы имеем дело с текстами.

## Задача: классификация твитов по тональности

У нас есть выборка из твитов.
Нам известна эмоциональная окраска каждого твита из выборки: положительная или отрицательная. Задача состоит в построении модели, которая по тексту твита предсказывает его эмоциональную окраску.

Классификацию по тональности используют в рекомендательных системах, чтобы понять, понравилось ли людям кафе, кино, etc.

Скачиваем выборку ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

In [1]:
import urllib.request

url1 = 'https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=1'
url2 = 'https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=1'

urllib.request.urlretrieve(url1, 'positive.csv')
urllib.request.urlretrieve(url2, 'negative.csv')


('negative.csv', <http.client.HTTPMessage at 0x7f2eacf2d1b0>)

In [2]:
import pandas as pd # библиотека для удобной работы с датафреймами
import numpy as np # библиотека для удобной работы со списками и матрицами
 
# библиотека, где реализованы основные алгоритмы машинного обучения
from sklearn.metrics import * 
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline

In [3]:
!head positive.csv

"408906692374446080";"1386325927";"pleease_shut_up";"@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)";"1";"0";"0";"0";"7569";"62";"61";"0"
"408906692693221377";"1386325927";"alinakirpicheva";"Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D";"1";"0";"0";"0";"11825";"59";"31";"2"
"408906695083954177";"1386325927";"EvgeshaRe";"RT @KatiaCheh: Ну ты идиотка) я испугалась за тебя!!!";"1";"0";"1";"0";"1273";"26";"27";"0"
"408906695356973056";"1386325927";"ikonnikova_21";"RT @digger2912: ""Кто то в углу сидит и погибает от голода, а мы ещё 2 порции взяли, хотя уже и так жрать не хотим"" :DD http://t.co/GqG6iuE2…";"1";"0";"1";"0";"1549";"19";"17";"0"
"408906761416867842";"1386325943";"JumpyAlex";"@irina_dyshkant Вот что значит страшилка :D
Но блин,посмотрев все части,у тебя создастся ощущение,что авторы курили что-то :D";"1";"0";"0";"0";"597";"16";"23";"1"
"408906761769598976";"1386325943";"JustinB94262583";"ну любишь и

Откроем файлы и создадим массив из текстов и правильных меток для твитов.
Сначала идут положительные твиты, потом отрицательные.

In [4]:
# загружаем положительные твиты
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text']) 
positive['label'] = ['positive'] * len(positive) # устанавливаем метки
 
# загружаем отрицательные твиты
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative) # устанавливаем метки
 
# соединяем вместе
df = pd.concat([positive, negative])

В этом коде происходит две вещи:

Сначала с помощью функции pd.read_csv из библиотеки pandas загружается CSV-файл с именем 'positive.csv'. Параметр sep=';' указывает, что в этом файле в качестве разделителя используется символ ';'. Параметр usecols=[3] говорит о том, что нужно загрузить только одну колонку с индексом 3 (индексация начинается с 0). Параметр names=['text'] задает имя этой колонке - 'text'. Результат загрузки сохраняется в переменной positive, которая теперь является DataFrame'ом.

Затем в этот DataFrame добавляется новая колонка с именем 'label'. Всем строкам этой колонки присваивается значение 'positive'. Это делается с помощью выражения ['positive'] * len(positive), которое создает список из строки 'positive', повторенной столько раз, сколько строк в DataFrame positive.

Посмотрим на полученные данные:

In [5]:
df.sample(5, random_state=40) # выводим 5 случайных примеров

,text,label
15931,RT @Blawar_1337: Теперь у нас с @Wake_UA появи...,positive
59532,с днём рождения зайка*))) ухх погуляем мы сего...,positive
47185,RT @Shumkova0406199: @ann_safina Вов вов вов А...,negative
42002,"Надо выдернуть звуковую дорожку из ""Доктора Ка...",positive
109035,@_hassliebe_ может все таки на этой неделе вер...,negative


Разбиваем данные на обучающую и тестовую выборки с помощью функции ```train_test_split()``` из **sklearn**:


In [6]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)
x_train

52772     хочу убивать людей, но меня останавливает УК Р...
16875     Я уже хочу говорить фразу: "За такие шутки, в ...
31371     @EugeneKarnak та фигня это репейное ( даже на ...
79679     @keinesommer @sladkobedrenna красивая упаковка...
55126     Некоторым в группе сделала подарки на рождеств...
                                ...                        
72867     @ecoprettiness что ты делаешь в четверг в 16:1...
24453     а вот в моём детстве нельзя было привлечь вним...
105206    @madya_kabutova дададааа как будто мы не знаем...
10098     теперь я могу разобрать горелку Kovea Mooonwak...
12258     Вот она "демократическая" поправка в Избирател...
Name: text, Length: 170125, dtype: object

## Baseline: классификация необработанных n-грамм

* Сейчас мы попробуем получить преобразование предложений в численный вектор, с которым может работать стандартный алгоритм машинного обучения, такой как логистическая регрессия. 
* Для этого нам понадобится познакомиться с понятием n-gram - самых мелких элементов предложения, с которыми можно работать. 
* Подсчитав количество этих n-грам в предложениях, мы получим искомые численные представления.

## Что такое n-граммы:

Самые мелкие структуры языка, с которыми мы работаем, называются **n-граммами**.
У n-граммы есть параметр n - количество слов, которые попадают в такое представление текста.
* Если n = 1 - то мы смотрим на то, сколько раз каждое слово встретилось в тексте. Получаем _униграммы_
* Если n = 2 - то мы смотрим на то, сколько раз каждая пара подряд идущих слов, встретилась в тексте. Получаем _биграммы_

Функция для работы с n-граммами реализована в библиотке **nltk** (Natural Language ToolKit), импортируем эту функцию: 

In [8]:
from nltk import ngrams

Прежде чем получать n-граммы, нужно разделить предложение на отдельные слова.  Для этого используем метод ```split()```.

In [9]:
sentence = 'Если б мне платили каждый раз'.split()
sentence

['Если', 'б', 'мне', 'платили', 'каждый', 'раз']

Чтобы получить n-грамму для такой последовательности, используем функцию ```ngrams()```. 

На вход передается два параметра:
* лист с разделенным на отдельные слова предложением (у нас он хранится в переменной ```sent```);
* параметр n, определяющий, какой тип n-грамм мы хотим получить.


Чтобы полученный объект отобразить, делаем из него ```list```. 

In [10]:
list(ngrams(sentence, 1)) # униграммы (1-граммы) - слова в предложении по отдельности (по одному слову) 

[('Если',), ('б',), ('мне',), ('платили',), ('каждый',), ('раз',)]

Функция ngrams из библиотеки NLTK используется для создания n-грамм из предложения. N-грамма - это последовательность из n слов.

В данном случае ngrams(sentence, 2) создает биграммы, то есть пары последовательных слов из предложения.

Предположим, у вас есть предложение "Я люблю программировать на Python". Биграммы этого предложения будут: ("Я", "люблю"), ("люблю", "программировать"), ("программировать", "на"), ("на", "Python").

Функция list используется для преобразования результата в список, так как ngrams возвращает генератор.



Аналогично мы можем получить биграммы - для этого заменяем параметр **n** в функции **ngrams** с 1 на 2.

In [11]:
list(ngrams(sentence, 2)) # биграммы

[('Если', 'б'),
 ('б', 'мне'),
 ('мне', 'платили'),
 ('платили', 'каждый'),
 ('каждый', 'раз')]

In [12]:
list(ngrams(sentence, 3)) # триграммы

[('Если', 'б', 'мне'),
 ('б', 'мне', 'платили'),
 ('мне', 'платили', 'каждый'),
 ('платили', 'каждый', 'раз')]

In [13]:
list(ngrams(sentence, 5)) # ... пентаграммы?

[('Если', 'б', 'мне', 'платили', 'каждый'),
 ('б', 'мне', 'платили', 'каждый', 'раз')]

### Векторизаторы

Векторизатор преобразует слово или набор слов в числовой вектор, понятный алгоритму машинного обучения, который привык работать с числовыми табличными данными.

Ниже - пример преобразования слов в двумерных вектор, каждому слову соответствует точка на плоскости.

<a href="https://drive.google.com/uc?id=1ukv-FTj0jeVdcgVlOaNBocUfNuYGGVZg
" target="_blank"><img src="https://drive.google.com/uc?id=1ukv-FTj0jeVdcgVlOaNBocUfNuYGGVZg" 
alt="IMAGE ALT TEXT HERE" width="600" border="0" /></a>

На начальном этапе нам будет достаточно тех инструментов, которые уже есть в знакомой нам библиотеке **sklearn**.

In [14]:
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer # модель "мешка слов", см. далее

Самый простой способ извлечь признаки из текстовых данных -- векторизаторы: `CountVectorizer` и `TfidfVectorizer`

Объект `CountVectorizer` делает следующую вещь:
* строит для каждого документа (каждой пришедшей ему строки) вектор размерности `n`, где `n` -- количество слов или n-грам во всём корпусе
* заполняет каждый i-тый элемент количеством вхождений слова в данный документ

<a href="https://drive.google.com/uc?id=1ukv-FTj0jeVdcgVlOaNBocUfNuYGGVZg
" target="_blank"><img src="https://drive.google.com/uc?id=1jHmkrGZTMawM46Yzxh243Ur1y5pYKzrl" 
alt="IMAGE ALT TEXT HERE" width="600" border="0" /></a>

На рисунке пример векторизации для униграмм, но можно использовать любые n-граммы. Для этого у объекта ```CountVectorizer()``` есть параметр **ngram_range**, который отвечает за то, какие n-граммы мы используем в качестве признаов:<br/>
ngram_range=(1, 1) -- униграммы<br/>
ngram_range=(3, 3) -- триграммы<br/>
ngram_range=(1, 3) -- униграммы, биграммы и триграммы.

<a href="https://drive.google.com/uc?id=1ODNVK0fdLTX4nv6ob55ciUe37d1pio-D" target="_blank"><img src="https://drive.google.com/uc?id=1ODNVK0fdLTX4nv6ob55ciUe37d1pio-D" 
alt="IMAGE ALT TEXT HERE" width="800" border="0" /></a>

Инициализируем ```CountVectorizer()```, указав в качестве признаков униграммы:

In [15]:
vectorizer = CountVectorizer(ngram_range=(1, 1)) # инициализируем "мешок слов" для униграмм 

Команда vectorizer = CountVectorizer(ngram_range=(1, 1)) создает объект CountVectorizer с параметром ngram_range=(1, 1).

CountVectorizer - это класс из библиотеки sklearn, который преобразует текстовые данные в векторы признаков. Каждому уникальному слову в тексте сопоставляется целочисленный идентификатор, а значение признака для данного слова в тексте равно количеству вхождений этого слова в текст.

Параметр ngram_range=(1, 1) указывает, что векторизатор должен учитывать только униграммы, то есть отдельные слова. Если бы вы хотели учитывать, например, биграммы (пары слов), вы бы использовали ngram_range=(1, 2).

Таким образом, эта команда создает объект CountVectorizer, который можно затем использовать для преобразования текстовых данных в векторы признаков, учитывая только униграммы.



После инициализации _vectorizer_ можно обучить на наших данных. 

Для обучения используем обучающую выборку ```x_train```, но в отличие от классификатора мы используем метод ```fit_transform()```: сначала обучаем наш векторизатор, а потом сразу применяем его к нашему набору данных. Это похоже на то, как мы работали с label encoderом и one-hot-encoderом.


In [17]:
vectorized_x_train = vectorizer.fit_transform(x_train)

In [18]:
vectorized_x_train

<170125x243804 sparse matrix of type '<class 'numpy.int64'>'
	with 1847487 stored elements in Compressed Sparse Row format>

Так как результат не зависит от порядка слов в текстах, то говорят, что такая модель представления текстов в виде векторов получается из *гипотезы представления текста как мешка слов*

"Гипотеза представления текста как мешка слов" (bag of words) - это подход к представлению текстовых данных для машинного обучения. В этом подходе текст представляется как мешок (или набор) слов, без учета грамматики и порядка слов, но с учетом частоты слов.

В контексте векторизации текста, как в вашем примере с CountVectorizer, каждое уникальное слово в тексте представляется в виде признака в векторе, а значение этого признака - это количество раз, которое слово встречается в тексте.

Таким образом, порядок слов в исходном тексте теряется, и текст превращается в набор слов (или "мешок слов"). Это может быть полезно для многих задач машинного обучения, где важна частота слов, а не их порядок.

В vectorizer.vocabulary_ лежит словарь, отображение слов в их индексы:

In [ ]:
list(vectorizer.vocabulary_.items())[:10]

[('rt', 74597),
 ('mr_malcev', 59533),
 ('пипец', 182397),
 ('завтро', 133145),
 ('первой', 179930),
 ('смены', 211532),
 ('daniilpanchenko', 23085),
 ('везет', 110835),
 ('нам', 163299),
 ('еще', 130686)]

В нашей выборке 170125 текстов (твитов), в них встречается 243760 разных слов.

In [ ]:
vectorized_x_train.shape

(170125, 243617)

Так как теперь у нас есть **численное представление** и набор входных признаков, то мы можем обучить модель логистической регрессии (или любую другую из тех, на которые мы смотрели раньше, например, случайный лес)

In [19]:
clf = LogisticRegression(random_state=42, max_iter=1000) # фиксируем random_state для воспроизводимости результатов
clf.fit(vectorized_x_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

С тестовыми данными нужно проделать то же самое, что и с данными для обучения: сделать из текстов вектора, которые можно передавать в классификатор для прогноза класса объекта. 

У нас уже есть обученный векторизатор ```vectorizer```, поэтому используем метод ```transform()``` (просто применить его), а не ```fit_transform``` (обучить и применить).

In [20]:
vectorized_x_test = vectorizer.transform(x_test)

Как раньше, для получения прогноза у обученного классификатора используем метод ```predict()```.

С помощью функции ```classification_report()```, которая считает сразу несколько метрик качества классификации, посмотрим на то, насколько хорошо мы предсказываем положительную или отрицательную тональность твита .

In [21]:
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       0.76      0.76      0.76     28019
    positive       0.77      0.77      0.77     28690

    accuracy                           0.77     56709
   macro avg       0.77      0.77      0.77     56709
weighted avg       0.77      0.77      0.77     56709



In [22]:
print(y_test.value_counts())

label
positive    28690
negative    28019
Name: count, dtype: int64


## N-граммы

Попробуем сделать то же самое, используя в качестве признаков триграммы:

In [23]:
# инициализируем векторайзер 
trigram_vectorizer = CountVectorizer(ngram_range=(3, 3))

# обучаем его и сразу применяем к x_train
trigram_vectorized_x_train = trigram_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(trigram_vectorized_x_train, y_train)

# применяем обученный векторизатор к тестовым данным
trigram_vectorized_x_test = trigram_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(trigram_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       0.72      0.47      0.57     28019
    positive       0.61      0.82      0.70     28690

    accuracy                           0.65     56709
   macro avg       0.67      0.64      0.63     56709
weighted avg       0.66      0.65      0.63     56709



Как вы думаете, почему в результатах теперь такой разброс по сравнению с униграммами?

Качество модели машинного обучения при использовании триграмм может быть хуже по сравнению с униграммами по нескольким причинам:

Разреженность данных: Триграммы создают гораздо больше уникальных признаков, многие из которых могут встречаться очень редко. Это может привести к "разреженности" данных, когда большинство признаков для каждого образца равны нулю. Модели могут быть менее эффективными при работе с такими разреженными данными.

Переобучение: Из-за большого количества признаков модель может "переобучиться" на тренировочных данных, то есть слишком хорошо подстроиться под них, учтя даже случайные шумы. В результате на тестовых данных качество может быть хуже.

Потеря важной информации: Триграммы учитывают контекст двух соседних слов, но при этом могут упустить более широкий контекст или важные отдельные слова. Униграммы, с другой стороны, не учитывают контекст, но могут лучше улавливать важность отдельных слов.

Все эти факторы могут влиять на качество модели. Оптимальный выбор между униграммами, биграммами, триграммами и т.д. зависит от конкретной задачи и данных.



## TF-IDF векторизация

`TfidfVectorizer` делает то же, что и `CountVectorizer`, но в качестве значений выдает **tf-idf** каждого слова.

Как считается tf-idf:

**TF (term frequency)** – относительная частотность слова в документе:
$$ TF(t,d) = \frac{n_{t}}{\sum_k n_{k}} $$

**IDF (inverse document frequency)** – обратная частота документов, в которых есть это слово:
$$ IDF(t, D) = \log \frac{|D|}{|{d : t \in d}|} $$

Перемножаем их:
$$TFIDF(t, d, D) = TF(t,d) \times IDF(i, D)$$

Сакральный смысл: если слово часто встречается в одном документе, но в целом по корпусу встречается в небольшом 
количестве документов, у него высокий TF-IDF.

Логический смысл TF-IDF следующий:

TF (Term Frequency): Если слово встречается часто в документе, то, вероятно, оно важно для понимания содержания этого документа. Поэтому мы увеличиваем вес этого слова пропорционально его частоте.

IDF (Inverse Document Frequency): Если слово встречается во многих документах, то, вероятно, оно менее дискриминативно или информативно для отдельного документа (например, общие слова типа "и", "в", "на"). Поэтому мы уменьшаем вес этого слова пропорционально числу документов, в которых оно встречается.

Таким образом, TF-IDF дает высокий вес словам, которые встречаются часто в данном документе, но редко в других документах, что делает их потенциально важными для понимания содержания данного документа.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

Действуем аналогично, как с ```CountVectorizer()```:

In [25]:
# инициализируем векторизатор, в качестве переменных используем униграммы
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 5))

# обучаем его и сразу применяем к x_train
tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(tfidf_vectorized_x_train, y_train)

# применяем обученный векторизатор к тестовым данным
tfidf_vectorized_x_test = tfidf_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(tfidf_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    negative       0.73      0.77      0.75     28019
    positive       0.77      0.72      0.74     28690

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.75     56709
weighted avg       0.75      0.75      0.75     56709



В этот раз получилось хуже :( Вернёмся к `CountVectorizer()`.

## Токенизация

Токенизировать - значит, поделить текст на части: слова, ключевые слова, фразы, символы и т.д., иными словами **токены**.

Самый наивный способ токенизировать текст - разделить с помощью функции `split()`. Но `split` упускает очень много всего, например, не отделяет пунктуацию от слов. Кроме этого, есть ещё много менее тривиальных проблем, поэтому лучше использовать готовые токенизаторы.

In [26]:
import nltk # уже знакомая нам библиотека nltk
from nltk.tokenize import word_tokenize # готовый токенизатор библиотеки nltk

Чтобы использовать токенизатор ```word_tokenize```, нужно сначала скачать данные для nltk о пунктуации и стоп-словах. Это просто требование nltk, поэтому, особо не задумываясь, запустите следующую ячейку:  

In [27]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Применим токенизацию:

In [28]:
example = 'Но не каждый хочет что-то исправлять:('
word_tokenize(example)

['Но', 'не', 'каждый', 'хочет', 'что-то', 'исправлять', ':', '(']

Если использовать просто ```split()```, то грустный смайлик :( не отделяется от слова "исправлять":

In [ ]:
example.split()

['Но', 'не', 'каждый', 'хочет', 'что-то', 'исправлять:(']

В nltk вообще есть довольно много токенизаторов:

In [29]:
from nltk import tokenize
dir(tokenize)[:16]

['BlanklineTokenizer',
 'LegalitySyllableTokenizer',
 'LineTokenizer',
 'MWETokenizer',
 'NLTKWordTokenizer',
 'PunktSentenceTokenizer',
 'RegexpTokenizer',
 'ReppTokenizer',
 'SExprTokenizer',
 'SpaceTokenizer',
 'StanfordSegmenter',
 'SyllableTokenizer',
 'TabTokenizer',
 'TextTilingTokenizer',
 'ToktokTokenizer',
 'TreebankWordDetokenizer']

Они умеют выдавать индексы в строке для начала и конца каждого слова-токена:

In [30]:
wh_tok = tokenize.WhitespaceTokenizer()
list(wh_tok.span_tokenize(example))

[(0, 2), (3, 5), (6, 12), (13, 18), (19, 25), (26, 38)]

Некторые токенизаторы ведут себя специфично:

In [31]:
tokenize.TreebankWordTokenizer().tokenize("don't stop me")

['do', "n't", 'stop', 'me']

А некоторые -- вообще не для текста на естественном языке:

In [32]:
tokenize.SExprTokenizer().tokenize("(a (b c)) d e (f)")

['(a (b c))', 'd', 'e', '(f)']

**Правильный токенизатор подбирается исходя из требований задачи!**

## Стоп-слова и пунктуация

**Стоп-слова** - это слова, которые часто встречаются практически в любом тексте и ничего интересного не говорят о конретном документе. Для модели это просто шум. А шум нужно убирать. По аналогичной причине убирают и пунктуацию.

In [33]:
# импортируем стоп-слова из библиотеки nltk
from nltk.corpus import stopwords

# посмотрим на стоп-слова для русского языка
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

*Знаки* пунктуации лучше импортировать из модуля **String**. В нем хранятся различные наборы констант для работы со строками (пунктуация, алфавит и др.). 

In [34]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

Объединим стоп-слова и знаки пунктуации вместе и запишем в переменную ```noise```:

In [35]:
noise = stopwords.words('russian') + list(punctuation)

Теперь нужно обучать нашу модель с учетом новых знаний про токенизацию и стоп-слова. 

Для этого мы можем собрать новый векторизатор, передав ему на вход:
* какие n-граммы нам нужны, параметр **ngram_range**;
* какой токенизатор мы используем, параметр **tokenizer**;
* какие у нас стоп-слова, параметр **stop_words**.

*Напоминание:* мы используем готовый токенизатор ```word_tokenize```, а стоп-слова хранятся в переменной ```noise```

In [36]:
# инициализируем умный векторайзер 
smart_vectorizer = CountVectorizer(ngram_range=(1, 1), 
                                   tokenizer=word_tokenize, 
                                   stop_words=noise)

In [37]:
# обучаем его и сразу применяем к x_train
smart_vectorized_x_train = smart_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = LogisticRegression(random_state=42)
clf.fit(smart_vectorized_x_train, y_train)

# применяем обученный векторайзер к тестовым данным
smart_vectorized_x_test = smart_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(smart_vectorized_x_test)
print(classification_report(y_test, pred))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  warnings.warn(


/home/codespace/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    negative       0.76      0.80      0.78     28019
    positive       0.79      0.76      0.78     28690

    accuracy                           0.78     56709
   macro avg       0.78      0.78      0.78     56709
weighted avg       0.78      0.78      0.78     56709



Получилось лучше: accuracy выше, а также заметно подрос recall у негативного класса. 

Что ещё можно сделать?

## Бонус*: Лемматизация

**Лемматизация** – это сведение разных форм одного слова к начальной форме – **лемме**. Почему это хорошо?
* Во-первых, естественно рассматривать как отдельный признак каждое *слово*, а не каждую его отдельную форму.
* Во-вторых, некоторые стоп-слова стоят только в начальной форме, и без лематизации выкидываем мы только её.

Для русского есть хороших лемматизатор pymorphy. 

### [Pymorphy](http://pymorphy2.readthedocs.io/en/latest/)
Это модуль на питоне, довольно быстрый и с кучей функций.

In [ ]:
# устанавливаем pymorphy2
!pip install pymorphy2

     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 8.2MB 7.4MB/s 


В pymorphy2 для морфологического анализа слов есть ```MorphAnalyzer()```:

In [38]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

pymorphy2 работает с отдельными словами. Если дать ему на вход предложение - он его просто не лемматизирует, т.к. не понимает:

In [39]:
sent = ['Если', 'б', 'мне', 'платили', 'каждый', 'раз']
sent

['Если', 'б', 'мне', 'платили', 'каждый', 'раз']

Лемматизируем слово "платили" из предложения ```sent``` с помощью метода ```parse()```:

In [47]:
ana = pymorphy2_analyzer.parse(sent[3])
ana

[Parse(word='платили', tag=OpencorporaTag('VERB,impf,tran plur,past,indc'), normal_form='платить', score=1.0, methods_stack=((DictionaryAnalyzer(), 'платили', 2472, 10),))]

Выведем его нормальную форму:

In [48]:
ana[0].normal_form

'платить'

## О важности эксплоративного анализа

Но иногда пунктуация бывает и не шумом - главное отталкиваться от задачи. Что будет если вообще не убирать пунктуацию?

In [49]:
# инициализируем умный векторайзер stop-words НЕ ИСПОЛЬЗУЕМ!
alternative_tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), 
                                               tokenizer=word_tokenize)

# обучаем его и сразу применяем к x_train
alternative_tfidf_vectorized_x_train = alternative_tfidf_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = LogisticRegression(random_state=42)
clf.fit(alternative_tfidf_vectorized_x_train, y_train)

# применяем обученный векторайзер к тестовым данным
alternative_tfidf_vectorized_x_test = alternative_tfidf_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(alternative_tfidf_vectorized_x_test)
print(classification_report(y_test, pred))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     28019
    positive       1.00      1.00      1.00     28690

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Шок! Стоило оставить пунктуацию -- и все метрики равны 1. Как это получилось? Среди неё были очень значимые токены (как вы думаете, какие?).

Посмотрим, как один из супер-значительных токенов справится с классификацией безо всякого машинного обучения:

In [ ]:
cool_token = ')'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92     32943
    positive       0.83      1.00      0.91     23766

    accuracy                           0.91     56709
   macro avg       0.91      0.93      0.91     56709
weighted avg       0.93      0.91      0.91     56709



## Символьные n-граммы

Теперь в качестве фичей используем, например, униграммы символов. Для этого необходимо установить в ```CountVectorizer()``` параметр ```analyzer = 'char'```, то есть анализировать символы.

In [ ]:
# инициализируем векторайзер для символов
char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(1, 1))

# обучаем его и сразу применяем к x_train
char_vectorized_x_train = char_vectorizer.fit_transform(x_train)

# инициализируем и обучаем классификатор
clf = LogisticRegression(random_state=42)
clf.fit(char_vectorized_x_train, y_train)

# применяем обученный векторайзер к тестовым данным
char_vectorized_x_test = char_vectorizer.transform(x_test)

# получаем предсказания и выводим информацию о качестве
pred = clf.predict(char_vectorized_x_test)
print(classification_report(y_test, pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              precision    recall  f1-score   support

    negative       1.00      0.99      1.00     28023
    positive       0.99      1.00      1.00     28686

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Из предыдущего раздела уже понятно, почему на этих данных точность равна 1. 

Символьные n-граммы используются, например, для задачи определения языка. Ещё одна замечательная особенность признаков-символов - для них не нужна токенизация и лемматизация, можно использовать такой подход для языков, у которых нет готовых анализаторов.

## Бонус***: регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [50]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
sent = 'hello world, I am Alina'.split()
for s in sent:
  print(re.findall('\w{2}',s))
#result=re.findall('\w{2}','hello world')
#result

['he', 'll']
['wo', 'rl']
[]
['am']
['Al', 'in']


### split
разделяет строку по заданному шаблону


In [53]:
result = re.split(',', 'itsy, bitsy, teenie, weenie')  # разбивает строку по разделителю ,
print(result)

['its', ', bits', ', teenie, weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [54]:
text = "Это первое предложение. Это второе предложение. Это третье предложение. Это четвертое предложение. Это пятое предложение."
sentences = re.split('\.', text)[:3] # разбиваем текст на предложения по точке и берем первые 3 предложения 
print(sentences)


['Это первое предложение', ' Это второе предложение', ' Это третье предложение']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc') # ищем все вхождения 'a' и заменяем их на 'b'
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [56]:
result = re.sub('\d', 'DIG', '1 2 3 5 f s s c5443 6')
print (result)

DIG DIG DIG DIG f s s cDIGDIGDIGDIG DIG


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [57]:
import re

emails = "abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz"

domains = re.findall(r'@\w+\.\w+', emails)

print(domains)


['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']


Если всё ещё осталось время: [регулярочный кроссворд ¯\_(ツ)_/¯](https://mariolurig.com/crossword/)